Ce notebook met en place la méthodologie pour appliquer le model DNN sur une nouvelle entreprise afin de trouver ces comparables.

Etant donnée une nouvelle entreprise cliente EC. L'algorithme doit:
1. extraire les features c.a.d les colonnes (Caractéristiques et indicateurs financiers ) qu'on va donner comme input à notre modèle, nous les appelorons caractéristiques_financiers.
2. Encodage des colonnes : 'Country', 'Sector' utilisons LabelEncoder ()
3. Normalization de tout les colonnes apart 'description', utilisant le RobustScaler()
4. Encodage de la colonne 'Description' utilisant la fonction vectorizer_coder du module my_function_vectorizer_coder_V9


### Packages 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, RobustScaler
import copy
import pickle
import plotly.express as px
from sklearn.metrics import r2_score

### Entrainnement et extraction des normalizeurs et encodeur (Ne toucher pas cette partie)


#### Entrainnement des encodeurs des colonnes 'Country' et 'Sector'

In [2]:
#Entrainnement des encodeurs
encoder_country = LabelEncoder().fit( data_V1.Country )
encoder_sector = LabelEncoder().fit ( data_V1.Sector )

NameError: name 'data_V1' is not defined

In [7]:
data_V2 = data_V1.copy ()
data_V2['Country'] = encoder_country.transform (data_V1.Country )
data_V2['Sector'] = encoder_sector.transform (data_V1.Sector)
data_V2

,Country,Sector,Net Debt / Equity (x) - Last,Net Sales - Last,ROA - Last,Sales To Assets (%) - Last,Sales To Receiv. - Last,Net Debt / Equity (x) - Last.1,Fin. Debt / Equity (%) - 2020,Fin. Debt / Assets (%) - 2020,...,Fin. Debt / Assets (%) - 2021.1,Fin. Debt / Assets (%) - Last.1,Current Ratio (%) - 2021,Current Ratio (%) - Last,Net Sales - Last.1,Days in Inventory - Last,Days in acc. Pay. - Last,Days in Receiv. - Last,Sales To Receiv. - Last.1,Description
0,68,64,-0.09,576.994052,5.62,197.58,22.97,-0.09,13.36,5.69,...,11.36,11.36,108.99,108.99,576.994052,27.11,27.76,17.95,22.97,"S&P Business Summary11 plc, together with its ..."
1,68,111,0.87,15.569810,-2.56,108.03,21.90,0.87,68.21,13.49,...,21.38,21.38,64.55,64.55,15.569810,42.33,63.77,13.19,21.90,S&P Business SummaryABC Transport Plc provides...
2,68,66,-0.10,3.263938,8.45,8.74,3.80,-0.10,0.00,0.00,...,0.00,0.00,222.58,222.58,3.263938,0.00,13.80,165.81,3.80,S&P Business SummaryAfrica Prudential Plc prov...
3,68,4,0.03,5.748802,1.24,88.86,960.15,0.03,0.00,0.00,...,0.00,3.59,272.66,244.25,5.748802,112.55,7.96,0.21,960.15,S&P Business SummaryAluminium Extrusion Indust...
4,68,64,5.75,524.974293,-5.96,179.15,23.41,5.75,49.70,13.81,...,48.88,41.02,112.35,90.79,524.974293,22.47,29.59,18.35,23.41,"S&P Business SummaryArdova Plc, an integrated ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21724,72,14,0.61,163.643593,-0.18,86.74,6.85,-0.65,0.00,0.00,...,17.25,27.41,64.68,54.60,100.417108,11.67,44.61,43.85,16.65,S&P Business SummaryMetro Performance Glass Li...
21725,72,5,-0.49,375.714408,9.17,116.44,130.56,-0.42,0.00,0.00,...,6.24,6.24,266.09,266.09,901.139196,35.41,21.01,17.01,25.10,S&P Business SummaryMichael Hill International...
21726,72,10,-0.94,0.000000,-23.98,0.00,0.00,-0.27,7.38,5.68,...,19.58,19.58,51.72,51.72,19.173878,15.52,30.07,37.46,8.67,S&P Business SummaryNeuren Pharmaceuticals Lim...
21727,72,40,-0.40,52.309191,8.06,29.68,17.41,0.81,103.49,46.54,...,13.59,13.59,714.33,714.33,84.715643,0.44,29.43,30.56,12.01,S&P Business SummaryNew Zealand Oil & Gas Limi...


In [54]:
#Enregistrement des encodeurs
with open ( 'Models/Encodeur_Country.pkl', 'wb' ) as f:
    pickle.dump( encoder_country, f )
with open ( 'Models/Encoder_Sector.pkl', 'wb' ) as f:
    pickle.dump ( encoder_sector, f )

#### Entrainnement du RobustScale() pour normalizer les données

In [8]:
#Entrainnement du normalizeur
robust_scale = RobustScaler().fit ( data_V2.drop('Description', axis = 1) )

In [9]:
data_V3 = robust_scale.transform (data_V2.drop('Description', axis = 1))
data_V3 = pd.DataFrame ( data_V3, columns = data_V2.drop('Description', axis = 1).columns )
data_V3

,Country,Sector,Net Debt / Equity (x) - Last,Net Sales - Last,ROA - Last,Sales To Assets (%) - Last,Sales To Receiv. - Last,Net Debt / Equity (x) - Last.1,Fin. Debt / Equity (%) - 2020,Fin. Debt / Assets (%) - 2020,...,Sales To Assets (%) - Last.1,Fin. Debt / Assets (%) - 2021.1,Fin. Debt / Assets (%) - Last.1,Current Ratio (%) - 2021,Current Ratio (%) - Last,Net Sales - Last.1,Days in Inventory - Last,Days in acc. Pay. - Last,Days in Receiv. - Last,Sales To Receiv. - Last.1
0,0.482759,0.134328,-0.135135,2.164286,0.327210,1.812357,2.086585,-0.211268,-0.212760,-0.357323,...,1.886934,-0.175800,-0.173529,-0.387253,-0.382711,1.519518,-0.224099,-0.172541,-0.574390,2.016949
1,0.482759,0.835821,1.162162,-0.125866,-0.611940,0.606946,1.956098,1.140845,0.584363,-0.076646,...,0.595472,0.192718,0.194853,-0.656489,-0.651734,-0.153099,-0.045524,0.524787,-0.641517,1.887409
2,0.482759,0.164179,-0.148649,-0.176064,0.652124,-0.729573,-0.251220,-0.225352,-0.406918,-0.562073,...,-0.836458,-0.593601,-0.591176,0.300921,0.304922,-0.189762,-0.542180,-0.442874,1.510788,-0.303874
3,0.482759,-0.761194,0.027027,-0.165928,-0.175660,0.348903,116.376829,-0.042254,-0.406918,-0.562073,...,0.319008,-0.593601,-0.459191,0.604326,0.436104,-0.182359,0.778364,-0.555964,-0.824566,115.476998
4,0.482759,0.134328,7.756757,1.952088,-1.002296,1.564275,2.140244,8.014085,0.315361,-0.065131,...,1.621142,1.204119,0.916912,-0.366897,-0.492887,1.364539,-0.278540,-0.137103,-0.568749,2.070218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21724,0.551724,-0.611940,0.810811,0.478154,-0.338691,0.320366,0.120732,-1.000000,-0.406918,-0.562073,...,-0.721517,0.040824,0.416544,-0.655701,-0.711968,0.099681,-0.405256,0.153757,-0.209138,1.251816
21725,0.551724,-0.746269,-0.675676,1.343229,0.734788,0.720151,15.207317,-0.676056,-0.406918,-0.562073,...,0.250937,-0.364104,-0.361765,0.564522,0.568315,2.485224,-0.126716,-0.303253,-0.587646,2.274818
21726,0.551724,-0.671642,-1.283784,-0.189378,-3.071183,-0.847220,-0.714634,-0.464789,-0.299666,-0.357683,...,-0.364148,0.126517,0.128676,-0.734218,-0.729403,-0.142362,-0.360084,-0.127808,-0.299253,0.285714
21727,0.551724,-0.223881,-0.554054,0.024001,0.607348,-0.447705,1.408537,1.056338,1.097079,1.112630,...,-0.801558,-0.093784,-0.091544,3.280141,3.281797,0.052903,-0.537017,-0.140201,-0.396559,0.690073


In [19]:
#Enregistrement du Normalizeur
with open ( 'Models/Normalizeur_RobustScaler.pkl', 'wb' ) as f:
    pickle.dump ( robust_scale, f )

### Importation des fichiers et modèles

In [2]:
#la data des comparables
entreprise_classe = pd.read_excel ( 'Data Finpack V2/entreprise_classe_V2_non_coder_non_normaliser.xlsx', index_col = 0 )
entreprise_classe

,Efcode,Company Name,Sector_Original,Country_Original,Description,EBITDA - Last,EBITDA Marg. (%) - Last,labels
9807,40838FC,Amulaire Thermal Technology Inc.,Heavy Construction,TWN,S&P Business SummaryAmulaire Thermal Technolog...,-5.999558,-25.11,0
9836,30071FC,Hocheng Corporation,Building Materials & Fixtures,TWN,"S&P Business SummaryHocheng Corporation, toget...",14.230506,8.70,0
9856,30823FC,Longda Construction & Development Corp.,Heavy Construction,TWN,S&P Business SummaryLongDa Construction & Deve...,34.956881,22.69,0
9885,30768FC,"Tung Kai Technology Engineering Co., Ltd.",Heavy Construction,TWN,S&P Business SummaryLung Ming Green Energy Tec...,-1.996294,-3.42,0
9941,30209FC,Apex Science & Engineering Corp.,Electronic Equipment,TWN,S&P Business SummaryApex Science & Engineering...,9.890537,10.25,0
...,...,...,...,...,...,...,...,...
83437,30345EN,Intertrust NV,Specialty Finance,NLD,S&P Business SummaryIntertrust N.V. provides c...,214.483968,33.03,13
83524,90006EN,Royal BAM Groep NV,Heavy Construction,NLD,"S&P Business SummaryKoninklijke BAM Groep nv, ...",375.143627,5.30,13
83525,90032EN,Royal Boskalis Westminster N.V.,Heavy Construction,NLD,S&P Business SummaryKoninklijke Boskalis N.V. ...,525.420933,15.63,13
83573,30261EN,Yandex NV,Internet,NLD,"S&P Business SummaryYandex N.V., a technology ...",613.900000,8.28,13


In [4]:
#Importation de la data initiale en cas de besoin
data = pd.read_excel ( 'Data Finpack V2/data_finpack_V2_sans_na_sans_Out.xlsx', index_col = 0 )
data

,Efcode,isActive,Country,Company Name,Sector,Description,Net Debt / Equity (x) - Last,Net Sales - Last,ROA - Last,Sales To Assets (%) - Last,...,Days in acc. Pay. - Last,Days in Receiv. - Last,Sales To Receiv. - Last.1,EBITDA - Last,Total Assets - Last,Sharehold. Equity - Last,EBITDA Marg. (%) - Last,EBIT Marg. (%) - Last,ROE (%) - Last,ROCE (%) - Last
22,30031KN,True,NGA,11,Integrated Oil & Gas,"S&P Business Summary11 plc, together with its ...",-0.09,576.994052,5.62,197.58,...,27.76,17.95,22.97,36.297033,292.032431,101.120098,6.29,4.07,14.77,17.25
23,30145KN,True,NGA,ABC Transport PLC,Travel & Tourism,S&P Business SummaryABC Transport Plc provides...,0.87,15.569810,-2.56,108.03,...,63.77,13.19,21.90,1.763778,14.412894,2.509889,11.33,5.56,-13.79,12.11
30,30268KN,True,NGA,Africa Prudential Plc,Investment Services,S&P Business SummaryAfrica Prudential Plc prov...,-0.10,3.263938,8.45,8.74,...,13.80,165.81,3.80,-0.030324,42.010095,20.460582,-0.73,-3.66,16.45,-3.09
35,30067KN,True,NGA,Aluminium Extrusion Industries PLC,Aluminum,S&P Business SummaryAluminium Extrusion Indust...,0.03,5.748802,1.24,88.86,...,7.96,0.21,960.15,0.244692,6.469231,5.307299,4.26,3.59,1.49,3.35
36,30033KN,True,NGA,Ardova Plc,Integrated Oil & Gas,"S&P Business SummaryArdova Plc, an integrated ...",5.75,524.974293,-5.96,179.15,...,29.59,18.35,23.41,3.180169,293.043683,20.217501,0.61,-1.03,-59.90,-3.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83676,30029AN,True,NZL,Millennium & Copthorne Hotels New Zealand Limited,Hotels,S&P Business SummaryMillennium & Copthorne Hot...,-0.10,91.536890,3.12,20.33,...,21.03,21.22,17.96,32.130402,450.191273,407.986595,35.10,29.99,3.45,8.81
83681,40127AN,True,NZL,NZ Automotive Investments Ltd.,Specialty Retailers,S&P Business SummaryNZ Automotive Investments ...,0.53,44.526214,7.20,170.81,...,0.00,59.82,12.20,4.297933,18.375558,10.120072,8.35,3.16,8.26,14.39
83687,30146AN,True,NZL,Neuren Pharmaceuticals Limited,Biotechnology,S&P Business SummaryNeuren Pharmaceuticals Lim...,-0.94,0.000000,-23.98,0.00,...,30.07,37.46,8.67,-0.902686,27.465892,26.400697,-9.77,-9.84,0.46,604.22
83690,30021AN,True,NZL,New Zealand Oil & Gas Limited,Exploration & Production,S&P Business SummaryNew Zealand Oil & Gas Limi...,-0.40,52.309191,8.06,29.68,...,29.43,30.56,12.01,24.768314,176.237520,101.366456,47.35,30.69,12.89,12.61


In [5]:
#Classification Features 
classification_features = ['Sector', 'Country', 'Description', 'EBITDA - Last', 'EBITDA Marg. (%) - Last']
classification_features

['Sector',
 'Country',
 'Description',
 'EBITDA - Last',
 'EBITDA Marg. (%) - Last']

In [6]:
#Importation du vocabulaire pour codage de la colonne description
vocabulaire_business = pd.read_csv ( 'Data Finpack/Vocabulaire company description.txt', sep= '\t' )
vocabulaire_business

,business,products,services,summary,listed,stock,short,company's,production,engaged,...,operational,manage,achieved,insights,vitro,machineries,thousand,pfts,intelligent,dairy


In [7]:
# Importation des encodeurs
with open ( 'Models_V2/country_encoder_data_ev.pkl', 'rb' ) as f:
    country_encoder = pickle.load ( f )


In [8]:
with open ( 'Models_V2/sector_encoder_data_ev.pkl', 'rb' ) as f:
    sector_encoder = pickle.load (f)

In [9]:
#Importation du Normalizeur (robust scaler)
with open ( 'Models_V2/normaliseur.pkl', 'rb' ) as f:
    normalizeur_RobustScaler = pickle.load (f) 

In [10]:
# Importation du model Réseau de neuronnes classificateur
with open ( 'Models_V2/Model_DNN_classificateur.pkl', 'rb' ) as f:
    model_DNN = pickle.load( f )
model_DNN

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-06-23 16:57:56         2485
metadata.json                                  2023-06-23 16:57:56           64
variables.h5                                   2023-06-23 16:57:56      5215240
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\dropout
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [11]:
#Importation des modèles prédicteur de EV/EBITDA, P/E, P/Book
with open ( 'Models_V2/Model_Gradient_Boosting_EV_EBITDA.pkl', 'rb' ) as f:
    model_EV_EBITDA = pickle.load (f)
with open ( 'Models_V2/Model_Gradient_Boosting_P_E.pkl', 'rb' ) as f:
    model_P_E = pickle.load (f)
with open ( 'Models_V2/Model_Gradient_Boosting_P_Book_value.pkl', 'rb' ) as f:
    model_P_Book = pickle.load (f)


In [12]:
#Importation de la fonction vectorizer_coder pour coder la colonne description
from my_function_vectorizer_coder_V9 import vectorizer_coder

### Application sur un exemple (Ne toucher pas cette partie)

In [67]:
#Choisissons un exemple i
efcode = entreprise_classe.Efcode.iloc [4454]
exemple = data[ data.Efcode == efcode ]
exemple 

,Efcode,isActive,Country,Company Name,Sector,Description,Net Debt / Equity (x) - Last,Net Sales - Last,ROA - Last,Sales To Assets (%) - Last,...,Days in acc. Pay. - Last,Days in Receiv. - Last,Sales To Receiv. - Last.1,EBITDA - Last,Total Assets - Last,Sharehold. Equity - Last,EBITDA Marg. (%) - Last,EBIT Marg. (%) - Last,ROE (%) - Last,ROCE (%) - Last
63697,80489FI,True,IND,UNIVERSAL AUTOFOUNDRY,Steel,S&P Business SummaryUniversal Autofoundry Limi...,2.16,21.258967,-3.82,182.75,...,72.64,0.0,4.09,3.110895,16.989855,7.062604,10.82,7.93,26.53,19.31


In [68]:
exemple_label = entreprise_classe [ entreprise_classe.Efcode == efcode ]
exemple_label 

,Efcode,Company Name,Sector_Original,Country_Original,Description,EBITDA - Last,EBITDA Marg. (%) - Last,labels
63697,80489FI,UNIVERSAL AUTOFOUNDRY,Steel,IND,S&P Business SummaryUniversal Autofoundry Limi...,3.110895,10.82,3


In [71]:
#1. 
exemple_V1 = exemple 

#2. Encodage des colonnes 'Country' et 'Sector'
exemple_V2 = exemple_V1.copy()
exemple_V2.Country = country_encoder.transform ( exemple_V2.Country )
exemple_V2.Sector = sector_encoder.transform ( exemple_V2.Sector )

#3. Normalisation des valeurs de l'exemple
exemple_V2 = exemple_V2.drop ( ['Efcode', 'isActive', 'Company Name', 'Description' ], axis =1 )
exemple_V3 = normalizeur_RobustScaler.transform ( exemple_V2 )
exemple_V3 = pd.DataFrame ( exemple_V3, columns = exemple_V2.columns )

#4. Selection des colonnes classification_features
exemple_V4 = exemple_V3[ ['Sector', 'Country',  'EBITDA - Last', 'EBITDA Marg. (%) - Last'] ]

#4. Codons la description de exemple
description_coder, description_vectorizer = vectorizer_coder (exemple.Description, vocabulaire= vocabulaire_business)

#5. Ajoutons la description coder à l'exemple 
exemple_V5 = pd.concat ([exemple_V4, description_coder], axis = 1 )
exemple_V5


Ajoutons n = len(colonne) lignes de valeurs 0 à la dataframe vocabulaire_business
Entrain d'appliquer le preprocessing à la colonne
voctorization de la description : 0
Entrain de transformer la colonne à une dataframe pour obtenir une colonne de description vectoriser
array to dataframe: ligne 0
Entrain de coder la nouvelle colonne vectoriser vers le vocabulaire
codage de la  0   ème ligne


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Sector,Country,EBITDA - Last,EBITDA Marg. (%) - Last,"(business,)","(products,)","(services,)","(summary,)","(listed,)","(stock,)",...,"(operational,)","(manage,)","(achieved,)","(insights,)","(vitro,)","(machineries,)","(thousand,)","(pfts,)","(intelligent,)","(dairy,)"
0,0.746269,0.0,-0.087607,-0.001213,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
#Prediction de la classe de l'exemple 
proba = model_DNN.predict ( exemple_V5 )
classe_predit = np.argmax (proba)
print ('''La classe de l'entreprise :''', exemple['Company Name'].iloc[0], ''' est :''', classe_predit)

1/1 [==============================] - 0s 31ms/step
La classe de l'entreprise : UNIVERSAL AUTOFOUNDRY  est : 3


### Test sur Echantillion du test

In [16]:
echantillon = pd.read_excel ( 'Data Finpack V2/Resultats_test_echantillon/Echantillon_.xlsx' )
echantillon

,Société,Secteur,Sector,Pays,Country,Description_V1,Description,Chiffre d'affaires (kMAD) (2022),EBITDA (kMAD) (2022),EBITDA - Last,Marge d'EBITDA,EBITDA Marg. (%) - Last,EV/EBITDA 2022,EV/EBITDA 2023,EV/EBITDA 2024,P/E 2023,P/E 2024
0,APDL,Avicole,Food Products,Maurice,MUS,Established in 2010 and incorporated in Maurit...,Established in 2010 and incorporated in Maurit...,1267509.07,1.886922e+05,19.293682,0.148869,14.886853,8.9,8.2,6.9,8.2,6.9
1,KEMIPHARM,Pharmaceutique,Pharmaceuticals,Maroc,MAR,Founded in 2012 and headquartered in Casablanc...,Founded in 2012 and headquartered in Casablanc...,70 962 (2024),1.567200e+04,1.602454,0.220000,22.000000,14.8,12.04,8.51,12.04,8.51
2,AL HANANE,Éducation et Formation,Education and Training,Maroc,MAR,Provides K-12 after-school tutoring services i...,AL HANANE is a distinguished educational insti...,77400,3.596000e+04,3.676892,0.465000,46.500000,n.d.,8.4,8.1,8.4,8.1
3,CECI,Poids lourd et carrosserie industrielle,Heavyweight and Industrial Bodywork (Trucks an...,Maroc,MAR,"Founded in 2004 and based in Casablanca, CECI ...","Founded in 2004 and based in Casablanca, CECI ...",187805,2.576200e+04,2.634151,0.137000,13.700000,n.d.,6.3,5.3,6.3,5.3
4,SGTM,Bâtiment et travaux publics,Heavy Construction,Maroc,MAR,Founded in 1971 by the brothers Ahmed and M'ha...,Founded in 1971 by the brothers Ahmed and M'ha...,7400000,6.860000e+05,70.143149,0.092703,9.270270,9.5,n.d.,n.d.,n.d.,n.d.
5,NOVACIM,Matériaux de construction,Building Materials & Fixtures,Maroc,MAR,Novacim is a Moroccan company specializing in ...,Novacim is a Moroccan company specializing in ...,582 612 (2024),2.411500e+05,24.657464,0.414000,41.400000,11.8,9.9,8.7,9.9,8.7
6,MERIDIAM (CBT),Transport collectif urbain,Transportation Services,Maroc,MAR,CityBus Transport Groupe is a 100% Moroccan-ow...,CityBus Transport Group is a holding company w...,334500,1.259000e+05,12.873211,0.376383,37.638266,5.1,n.d.,n.d.,n.d.,n.d.
7,IWACO,Distribution de produits de télécommunication ...,Telecommunications Equipment,Maroc,MAR,Founded in 2006 and headquartered in Casablanc...,Founded in 2006 and headquartered in Casablanc...,117 000 (2020),2.340000e+04,2.392638,0.200000,20.000000,6.75,6.72,6.1,6.72,6.1
8,PATISEN,Agroalimentaire,Food Products,Sénégal,CIV,"Founded in 1981, Patisen is the leading player...","Founded in 1981, Patisen is the leading player...",1267407.54,7.518500e+04,7.687628,0.059322,5.932188,9.3,4.4,n.d.,4.4,n.d.
9,Bank Of Africa,Banque et Finance,Banks,Maroc,MAR,"Founded in 1959 and privatized in 1995, Bank o...","Founded in 1959 and privatized in 1995, Bank o...",12058179,3.487343e+06,356.579039,0.289200,28.920000,n.a.,"9,7 ""P/E""","9,12 ""P/E""",9.7,9.12


In [17]:
#Choix des variables de classification 
echantillon_1 = echantillon [classification_features]
echantillon_1

,Sector,Country,Description,EBITDA - Last,EBITDA Marg. (%) - Last
0,Food Products,MUS,Established in 2010 and incorporated in Maurit...,19.293682,14.886853
1,Pharmaceuticals,MAR,Founded in 2012 and headquartered in Casablanc...,1.602454,22.000000
2,Education and Training,MAR,AL HANANE is a distinguished educational insti...,3.676892,46.500000
3,Heavyweight and Industrial Bodywork (Trucks an...,MAR,"Founded in 2004 and based in Casablanca, CECI ...",2.634151,13.700000
4,Heavy Construction,MAR,Founded in 1971 by the brothers Ahmed and M'ha...,70.143149,9.270270
5,Building Materials & Fixtures,MAR,Novacim is a Moroccan company specializing in ...,24.657464,41.400000
6,Transportation Services,MAR,CityBus Transport Group is a holding company w...,12.873211,37.638266
7,Telecommunications Equipment,MAR,Founded in 2006 and headquartered in Casablanc...,2.392638,20.000000
8,Food Products,CIV,"Founded in 1981, Patisen is the leading player...",7.687628,5.932188
9,Banks,MAR,"Founded in 1959 and privatized in 1995, Bank o...",356.579039,28.920000


In [18]:
#Supprimons les deux lignes dont le secteur n'est pas connu pour l'encodeur 
echantillon_2 = echantillon_1.drop ( [2, 3], axis = 0 )
echantillon_2

,Sector,Country,Description,EBITDA - Last,EBITDA Marg. (%) - Last
0,Food Products,MUS,Established in 2010 and incorporated in Maurit...,19.293682,14.886853
1,Pharmaceuticals,MAR,Founded in 2012 and headquartered in Casablanc...,1.602454,22.000000
4,Heavy Construction,MAR,Founded in 1971 by the brothers Ahmed and M'ha...,70.143149,9.270270
5,Building Materials & Fixtures,MAR,Novacim is a Moroccan company specializing in ...,24.657464,41.400000
6,Transportation Services,MAR,CityBus Transport Group is a holding company w...,12.873211,37.638266
7,Telecommunications Equipment,MAR,Founded in 2006 and headquartered in Casablanc...,2.392638,20.000000
8,Food Products,CIV,"Founded in 1981, Patisen is the leading player...",7.687628,5.932188
9,Banks,MAR,"Founded in 1959 and privatized in 1995, Bank o...",356.579039,28.920000
10,Real Estate Services,FRA,"Unibail-Rodamco-Westfield is an owner, develop...",568.203558,67.481349
11,Hotels,FRA,Accor SA operates a chain of hotels worldwide....,1437.981983,31.586961


In [19]:
#2. Encodage des colonnes 'Country' et 'Sector'
echantillon_3 = echantillon_2.copy()
echantillon_3.Country = country_encoder.transform ( echantillon_3.Country )
echantillon_3.Sector = sector_encoder.transform ( echantillon_3.Sector )
echantillon_3

,Sector,Country,Description,EBITDA - Last,EBITDA Marg. (%) - Last
0,44,82,Established in 2010 and incorporated in Maurit...,19.293682,14.886853
1,83,73,Founded in 2012 and headquartered in Casablanc...,1.602454,22.000000
4,55,73,Founded in 1971 by the brothers Ahmed and M'ha...,70.143149,9.270270
5,14,73,Novacim is a Moroccan company specializing in ...,24.657464,41.400000
6,112,73,CityBus Transport Group is a holding company w...,12.873211,37.638266
7,108,73,Founded in 2006 and headquartered in Casablanc...,2.392638,20.000000
8,44,22,"Founded in 1981, Patisen is the leading player...",7.687628,5.932188
9,9,73,"Founded in 1959 and privatized in 1995, Bank o...",356.579039,28.920000
10,91,37,"Unibail-Rodamco-Westfield is an owner, develop...",568.203558,67.481349
11,59,37,Accor SA operates a chain of hotels worldwide....,1437.981983,31.586961


In [20]:
#3. Normalisation des valeurs de l'exemple
echantillon_4 = echantillon_3.drop ( ['Description'], axis = 1 )
echantillon_5 = normalizeur_RobustScaler.transform ( echantillon_4 )
echantillon_5 = pd.DataFrame ( echantillon_5, columns = echantillon_4.columns )
echantillon_5

,Sector,Country,EBITDA - Last,EBITDA Marg. (%) - Last
0,-0.164179,0.767857,0.312419,0.245413
1,0.417910,0.607143,-0.124894,0.676774
2,0.000000,0.607143,1.569377,-0.095193
3,-0.611940,0.607143,0.445007,1.853244
4,0.850746,0.607143,0.153710,1.625122
5,0.791045,0.607143,-0.105362,0.555488
6,-0.164179,-0.303571,0.025526,-0.297624
7,-0.686567,0.607143,8.649845,1.096422
8,0.537313,-0.035714,13.881035,3.434891
9,0.059701,-0.035714,35.381268,1.258154


In [21]:
#4. Codons la description 
description_coder, description_vectorizer = vectorizer_coder (echantillon.Description, vocabulaire= vocabulaire_business)
description_coder

Ajoutons n = len(colonne) lignes de valeurs 0 à la dataframe vocabulaire_business


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Entrain d'appliquer le preprocessing à la colonne
voctorization de la description : 0
voctorization de la description : 1
voctorization de la description : 2
voctorization de la description : 3
voctorization de la description : 4
voctorization de la description : 5
voctorization de la description : 6
voctorization de la description : 7
voctorization de la description : 8
voctorization de la description : 9
voctorization de la description : 10
voctorization de la description : 11
voctorization de la description : 12
voctorization de la description : 13
voctorization de la description : 14
Entrain de transformer la colonne à une dataframe pour obtenir une colonne de description vectoriser
array to dataframe: ligne 0
array to dataframe: ligne 1
array to dataframe: ligne 2
array to dataframe: ligne 3
array to dataframe: ligne 4
array to dataframe: ligne 5
array to dataframe: ligne 6
array to dataframe: ligne 7
array to dataframe: ligne 8
array to dataframe: ligne 9
array to dataframe: lign

,business,products,services,summary,listed,stock,short,company's,production,engaged,...,operational,manage,achieved,insights,vitro,machineries,thousand,pfts,intelligent,dairy
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#5. Ajoutons la description coder à l'exemple 
echantillon_6 = pd.merge (echantillon_5, description_coder, right_index = True, left_index = True )
echantillon_6

,Sector,Country,EBITDA - Last,EBITDA Marg. (%) - Last,"(business,)","(products,)","(services,)","(summary,)","(listed,)","(stock,)",...,"(operational,)","(manage,)","(achieved,)","(insights,)","(vitro,)","(machineries,)","(thousand,)","(pfts,)","(intelligent,)","(dairy,)"
0,-0.164179,0.767857,0.312419,0.245413,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.417910,0.607143,-0.124894,0.676774,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.607143,1.569377,-0.095193,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.611940,0.607143,0.445007,1.853244,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.850746,0.607143,0.153710,1.625122,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.791045,0.607143,-0.105362,0.555488,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.164179,-0.303571,0.025526,-0.297624,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-0.686567,0.607143,8.649845,1.096422,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.537313,-0.035714,13.881035,3.434891,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.059701,-0.035714,35.381268,1.258154,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#Prediction de la classe de l'echantillon
proba = model_DNN.predict ( echantillon_6 )
classe_predit = np.argmax ( proba, axis = 1 )
classe_predit

1/1 [==============================] - 0s 368ms/step


array([12, 12, 12,  7, 12, 12, 12, 13, 13,  6,  5,  5,  5], dtype=int64)

In [26]:
#Ajoutons les classes des entreprises à la data
echantillon_2_classer = echantillon_2.copy()
echantillon_2_classer ['Classe'] = classe_predit
echantillon_2_classer = pd.merge ( echantillon['Société'], echantillon_2_classer , right_index = True, left_index = True )
echantillon_2_classer

,Société,Sector,Country,Description,EBITDA - Last,EBITDA Marg. (%) - Last,Classe
0,APDL,Food Products,MUS,Established in 2010 and incorporated in Maurit...,19.293682,14.886853,12
1,KEMIPHARM,Pharmaceuticals,MAR,Founded in 2012 and headquartered in Casablanc...,1.602454,22.000000,12
4,SGTM,Heavy Construction,MAR,Founded in 1971 by the brothers Ahmed and M'ha...,70.143149,9.270270,12
5,NOVACIM,Building Materials & Fixtures,MAR,Novacim is a Moroccan company specializing in ...,24.657464,41.400000,7
6,MERIDIAM (CBT),Transportation Services,MAR,CityBus Transport Group is a holding company w...,12.873211,37.638266,12
7,IWACO,Telecommunications Equipment,MAR,Founded in 2006 and headquartered in Casablanc...,2.392638,20.000000,12
8,PATISEN,Food Products,CIV,"Founded in 1981, Patisen is the leading player...",7.687628,5.932188,12
9,Bank Of Africa,Banks,MAR,"Founded in 1959 and privatized in 1995, Bank o...",356.579039,28.920000,13
10,Unibail-Rodamco-Westfield,Real Estate Services,FRA,"Unibail-Rodamco-Westfield is an owner, develop...",568.203558,67.481349,13
11,ACCOR,Hotels,FRA,Accor SA operates a chain of hotels worldwide....,1437.981983,31.586961,6


In [87]:
echantillon_2_classer.to_excel ('Data Finpack V2/Echantillon_test_classer.xlsx')

#### Prediction de EV/EBITDA

In [23]:
#Data
echantillon_2_classer = pd.read_excel ( 'Data Finpack V2/Echantillon_test_classer.xlsx', index_col= 0 )
echantillon_2_classer

,Société,Sector,Country,Description,EBITDA - Last,EBITDA Marg. (%) - Last,Classe
0,APDL,Food Products,MUS,Established in 2010 and incorporated in Maurit...,19.293682,14.886853,12
1,KEMIPHARM,Pharmaceuticals,MAR,Founded in 2012 and headquartered in Casablanc...,1.602454,22.000000,12
4,SGTM,Heavy Construction,MAR,Founded in 1971 by the brothers Ahmed and M'ha...,70.143149,9.270270,12
5,NOVACIM,Building Materials & Fixtures,MAR,Novacim is a Moroccan company specializing in ...,24.657464,41.400000,12
6,MERIDIAM (CBT),Transportation Services,MAR,CityBus Transport Groupe is a 100% Moroccan-ow...,12.873211,37.638266,12
7,IWACO,Telecommunications Equipment,MAR,Founded in 2006 and headquartered in Casablanc...,2.392638,20.000000,12
8,PATISEN,Food Products,CIV,"Founded in 1981, Patisen is the leading player...",7.687628,5.932188,12
9,Bank Of Africa,Banks,MAR,"Founded in 1959 and privatized in 1995, Bank o...",356.579039,28.920000,13
10,Unibail-Rodamco-Westfield,Real Estate Services,FRA,The Group operates commercial real estate asse...,568.203558,67.481349,13
11,ACCOR,Hotels,FRA,The Group provides hotels and tourism services...,1437.981983,31.586961,6


In [27]:
data = pd.read_excel ( 'data Finpack V2/data_ev_ebitda_sans_na_sans_out_coder_normalizer_classer.xlsx', index_col = 0 )
data

,Efcode,Country_Original,Company Name,Sector_Original,Description,Sector,Country,EBITDA - Last,EBITDA Marg. (%) - Last,Classe,EV/EBITDA - Last
23,30145KN,NGA,ABC Transport PLC,Travel & Tourism,S&P Business SummaryABC Transport Plc provides...,0.865672,0.839286,-0.120907,0.029715,4,2.62
30,30268KN,NGA,Africa Prudential Plc,Investment Services,S&P Business SummaryAfrica Prudential Plc prov...,0.164179,0.839286,-0.165255,-0.701637,4,34.04
36,30033KN,NGA,Ardova Plc,Integrated Oil & Gas,"S&P Business SummaryArdova Plc, an integrated ...",0.134328,0.839286,-0.085894,-0.620376,4,54.07
39,30156KN,NGA,BOC Gases Plc,Medical Equipment,S&P Business SummaryIndustrial and Medical Gas...,0.238806,0.839286,-0.089054,0.933293,4,0.97
41,30291KN,NGA,BUA Cement Plc,Building Materials & Fixtures,S&P Business SummaryBUA Cement Plc engages in ...,-0.611940,0.839286,8.381335,2.006671,13,22.10
...,...,...,...,...,...,...,...,...,...,...,...
83776,40026AN,NZL,Vista Group International Ltd.,Software,S&P Business SummaryVista Group International ...,0.686567,0.928571,-0.049956,-0.329897,2,40.85
83777,30054AN,NZL,Vital Healthcare Property Trust,Speciality REITs,S&P Business SummaryVital Healthcare Property ...,0.701493,0.928571,1.154673,2.952699,7,29.59
83778,30160AN,NZL,Vital Limited,Mobile Telecommunications,S&P Business SummaryVital Limited provides mob...,0.268657,0.928571,0.007774,1.546998,4,2.30
83783,30035AN,NZL,Warehouse Group Ltd.,Broadline Retailers,S&P Business SummaryThe Warehouse Group Limite...,-0.626866,0.928571,4.574564,-0.095816,5,2.24


In [28]:
#Fixant l'intervalle de EV/EBITDA
data = data [ (data['EV/EBITDA - Last']>= 3) & (data['EV/EBITDA - Last']<= 50) ]
data

,Efcode,Country_Original,Company Name,Sector_Original,Description,Sector,Country,EBITDA - Last,EBITDA Marg. (%) - Last,Classe,EV/EBITDA - Last
30,30268KN,NGA,Africa Prudential Plc,Investment Services,S&P Business SummaryAfrica Prudential Plc prov...,0.164179,0.839286,-0.165255,-0.701637,4,34.04
41,30291KN,NGA,BUA Cement Plc,Building Materials & Fixtures,S&P Business SummaryBUA Cement Plc engages in ...,-0.611940,0.839286,8.381335,2.006671,13,22.10
47,30027KN,NGA,Cadbury Nigeria PLC,Food Products,S&P Business SummaryCadbury Nigeria Plc manufa...,-0.164179,0.839286,-0.075901,-0.476653,4,11.55
50,30275KN,NGA,Caverton Offshore Support Group Plc,Oil Equipment & Services,S&P Business SummaryCaverton Offshore Support ...,0.373134,0.839286,0.387410,0.986052,4,3.06
52,30216KN,NGA,Champion Breweries PLC,Brewers,S&P Business SummaryChampion Breweries Plc bre...,-0.656716,0.839286,0.038386,1.200728,4,12.16
...,...,...,...,...,...,...,...,...,...,...,...
83768,30083AN,NZL,Tourism Holdings Limited,Travel & Tourism,"S&P Business SummaryTourism Holdings Limited, ...",0.865672,0.928571,0.620521,0.235294,4,17.93
83774,30057AN,NZL,Turners Automotive Group Limited,Investment Services,S&P Business SummaryTurners Automotive Group L...,0.164179,0.928571,0.193736,-0.229230,4,13.29
83775,30123AN,NZL,Vector Limited,Multiutilities,"S&P Business SummaryVector Limited, together w...",0.313433,0.928571,10.155944,2.372347,13,6.05
83776,40026AN,NZL,Vista Group International Ltd.,Software,S&P Business SummaryVista Group International ...,0.686567,0.928571,-0.049956,-0.329897,2,40.85


In [31]:
#Les socièté
echantillon_2_classer ['Société']

0                              APDL
1                         KEMIPHARM
4                              SGTM
5                           NOVACIM
6                    MERIDIAM (CBT)
7                             IWACO
8                           PATISEN
9                    Bank Of Africa
10        Unibail-Rodamco-Westfield
11                            ACCOR
12                  Centrale Danone
13                   Groupe Akdital
14    Six Flags Entertainment Corp.
Name: Société, dtype: object

In [41]:
#Commencons par l'entreprise APDL
#selection de la classe
company = 'APDL'
classe_ = echantillon_2_classer[echantillon_2_classer ['Société'] == company ].Classe.iloc[0]
classe = data[data.Classe == classe_]
print ( 'la classe de ', company , 'est ', classe_ )
classe

la classe de  APDL est  12


,Efcode,Country_Original,Company Name,Sector_Original,Description,Sector,Country,EBITDA - Last,EBITDA Marg. (%) - Last,Classe,EV/EBITDA - Last
110,30085KN,NGA,Livestock Feeds PLC,Farming & Fishing,S&P Business SummaryLivestock Feeds Plc manufa...,-0.208955,0.839286,-0.101205,-0.209824,12,9.47
288,40266SN,NOR,Etman International ASA,Electrical Components & Equipment,"S&P Business SummaryEtman International ASA, t...",-0.298507,0.892857,-0.083792,-0.263190,12,6.28
299,40352SN,NOR,Gfjord Invest AS,Electricity,S&P Business SummaryGFjord Invest AS manufactu...,-0.283582,0.892857,-0.141343,1.555488,12,3.38
326,40316SN,NOR,Huddly AS,Specialty Retailers,"S&P Business SummaryHuddly AS, a technology co...",0.761194,0.892857,-0.081294,-0.214069,12,12.78
328,40284SN,NOR,Hval Sjokoladefabrikk ASA,Food Retailers & Wholesalers,S&P Business SummaryHval Sjokoladefabrikk ASA ...,-0.149254,0.892857,-0.137426,0.300788,12,10.44
...,...,...,...,...,...,...,...,...,...,...,...
83339,30203EN,NLD,Alumexx N.V.,Computer Services,S&P Business SummaryAlumexx N.V. manufactures ...,-0.492537,0.875000,-0.155882,-0.412978,12,13.31
83366,X0003EN,NLD,Cabka N.V.,Investment Services,"S&P Business SummaryAs of February 28, 2022, D...",0.164179,0.875000,-0.531932,-1.060643,12,8.04
83596,30107AN,NZL,Argosy Property Limited,Diversified REITs,S&P Business SummaryArgosy Property Limited in...,-0.343284,0.928571,-0.581089,-1.787144,12,17.51
83597,40036AN,NZL,"Arvida Group Ltd,",Health Care Providers,"S&P Business SummaryArvida Group Limited, toge...",-0.014925,0.928571,0.297905,0.201334,12,38.49


In [50]:
#Selection du secteur
secteur = echantillon_2_classer [echantillon_2_classer ['Société'] == company].Sector.iloc[0]
classe_12_secteur_food = classe [ classe.Sector_Original == secteur ]
#data = classe_12_secteur_food.copy()
print ( 'Taille : ', classe_12_secteur_food.shape)
#Listes des comparbles de la société
print ('Liste des comprables de :', company)
classe_12_secteur_food

Taille :  (53, 11)
Liste des comprables de : APDL


,Efcode,Country_Original,Company Name,Sector_Original,Description,Sector,Country,EBITDA - Last,EBITDA Marg. (%) - Last,Classe,EV/EBITDA - Last
700,30228FP,PAK,Al-Noor Sugar Mills Limited,Food Products,S&P Business SummaryAl-Noor Sugar Mills Limite...,-0.164179,0.964286,-0.033668,0.043663,12,5.22
801,30236FP,PAK,Dewan Sugar Mills Limited,Food Products,S&P Business SummaryDewan Sugar Mills Limited ...,-0.164179,0.964286,-0.144001,-0.520315,12,17.92
830,30260FP,PAK,Faran Sugar Mills Ltd,Food Products,"S&P Business SummaryFaran Sugar Mills Limited,...",-0.164179,0.964286,-0.082943,0.033960,12,7.06
953,30217FP,PAK,Khairpur Sugar Mills Ltd.,Food Products,S&P Business SummaryKhairpur Sugar Mills Limit...,-0.164179,0.964286,-0.089324,0.030321,12,3.86
966,30192FP,PAK,Leiner Pak Gelatine Limited,Food Products,S&P Business SummaryLeiner Pak Gelatine Limite...,-0.164179,0.964286,-0.156305,-0.247423,12,5.58
990,30409FP,PAK,Mirpurkhas Sugar Mills Limited,Food Products,S&P Business SummaryMirpurkhas Sugar Mills Lim...,-0.164179,0.964286,-0.078497,0.348696,12,8.13
1078,30659FP,PAK,"Premier Sugar Mills & Distillery Co, Ltd,",Food Products,S&P Business SummaryThe Premier Sugar Mills & ...,-0.164179,0.964286,0.117072,0.241358,12,7.24
1136,30123FP,PAK,Shahtaj Sugar Mills Limited,Food Products,S&P Business SummaryShahtaj Sugar Mills Limite...,-0.164179,0.964286,-0.107057,-0.166768,12,8.45
1176,30158FP,PAK,Tandlianwala Sugar Mills Limited,Food Products,S&P Business SummaryTandlianwala Sugar Mills L...,-0.164179,0.964286,0.210376,-0.001213,12,6.13
2653,40741EO,POL,M FOOD SA,Food Products,S&P Business SummaryM Food S.A. provides honey...,-0.164179,1.071429,-0.129502,-0.462098,12,5.15


In [295]:
classe_12_secteur_food.to_excel ( 'Data Finpack V2/resultats_test_echantillon/comparables_Six Flags Entertainment Corp..xlsx' )

In [46]:
#Loop over classes
data_finale = pd.DataFrame (index = [0])
R = []
#Réduction des valeurs de P/E  3  <= P/E <=  50
data_ = data.loc [ (data['EV/EBITDA - Last'] <= 50) & (data['EV/EBITDA - Last'] >= 3) ]

#for classe in data_.Classe.unique():
    #Selectionner la classe 
    #data_classe = data_ [data_.Classe == classe]
 

#Loop over Sector
#for secteur in data_classe.Sector_Original.unique():

    #data_secteur = data_classe [data_classe ['Sector_Original'] == secteur]

    #Loop over EV/EBITDA values
for i in range (3, 27, 6):
    
    if i <= 27:
        EV_EBITDA = classe_12_secteur_food.loc[( classe_12_secteur_food['EV/EBITDA - Last'] < i+6) &  (classe_12_secteur_food['EV/EBITDA - Last'] >= i )] 

    r = len(EV_EBITDA) - len(EV_EBITDA)%9

    #Loop over company samples
    for i in range ( 0, r, 9 ):
        entreprises =  EV_EBITDA[ i:i+9 ]
            
        #Construction de la première ligne
        classe_secteur_ev_ebitda_entreprise = pd.DataFrame ( { 'EV/EBITDA0' : entreprises['EV/EBITDA - Last'].iloc[0] }, index = [0] )
        for j in range (1, 9):
            classe_secteur_ev_ebitda_entreprise ['EV/EBITDA' + str(j)] = entreprises['EV/EBITDA - Last'].iloc[j]
        classe_secteur_ev_ebitda_entreprise ['Sector_Original'] = classe_12_secteur_food.Sector_Original.iloc[0]
        classe_secteur_ev_ebitda_entreprise ['Sector'] = entreprises['Sector'].iloc[0]
        classe_secteur_ev_ebitda_entreprise ['Classe'] = entreprises['Classe'].iloc[0]

        data_finale = pd.concat ( [data_finale, classe_secteur_ev_ebitda_entreprise] )

#Si ev/ebitda >= 27
EV_EBITDA = classe_12_secteur_food.loc[( classe_12_secteur_food['EV/EBITDA - Last'] <= 50) &  (classe_12_secteur_food['EV/EBITDA - Last'] >= 28 )] 
r = len(EV_EBITDA) - len(EV_EBITDA)%9

#Loop over company samples
for i in range ( 0, r, 9 ):
    entreprises =  EV_EBITDA[ i:i+9 ]
        
    #Construction de la première ligne
    classe_secteur_ev_ebitda_entreprise = pd.DataFrame ( { 'EV/EBITDA0' : entreprises['EV/EBITDA - Last'].iloc[0] }, index = [0] )
    for j in range (1, 9):
        classe_secteur_ev_ebitda_entreprise ['EV/EBITDA' + str(j)] = entreprises['EV/EBITDA - Last'].iloc[j]
    classe_secteur_ev_ebitda_entreprise ['Sector_Original'] = classe_12_secteur_food.Sector_Original.iloc[0]
    classe_secteur_ev_ebitda_entreprise ['Sector'] = entreprises['Sector'].iloc[0]
    classe_secteur_ev_ebitda_entreprise ['Classe'] = entreprises['Classe'].iloc[0]

    data_finale = pd.concat ( [data_finale, classe_secteur_ev_ebitda_entreprise] )

    #print ('Terminé pour secteur :', classe_12_secteur_food.Sector_Original)
#print ('Terminé pour classe =', classe)
data_finale     

#----------------------------------------------------------------------------------------------------------------------
#Supprimer la première ligne 
ev_ebitda_df = data_finale.copy()
#Initialisation des index
ev_ebitda_df = ev_ebitda_df.reset_index (drop = True)
#Supression de la première ligne
ev_ebitda_df = ev_ebitda_df.drop ( ev_ebitda_df.index[0] )
ev_ebitda_df



#Changement des places des colonnes 
sector_classe = ev_ebitda_df [['Sector_Original', 'Classe']]
ev_ebitda_df_V1 = ev_ebitda_df.drop (['Sector_Original', 'Sector', 'Classe'], axis = 1)
ev_ebitda_df_V1 = pd.merge ( sector_classe, ev_ebitda_df_V1,   right_index = True, left_index = True)
ev_ebitda_df_V1

#Codage de la colonne Sector_Original
with open ( 'Models_V2/sector_encoder_data_ev.pkl', 'rb' ) as f:
    sector_encoder = pickle.load (f)

ev_ebitda_df_V2 = ev_ebitda_df_V1.copy ()
ev_ebitda_df_V2['Sector'] = sector_encoder.transform (ev_ebitda_df_V2['Sector_Original'])
sector = ev_ebitda_df_V2 [['Sector']]
ev_ebitda_df_V2 = ev_ebitda_df_V2.drop ( ['Sector', 'Sector_Original'], axis = 1 )
ev_ebitda_df_V2 = pd.merge ( sector, ev_ebitda_df_V2,   right_index = True, left_index = True)
ev_ebitda_df_V2

#Prediction de EV/EBITDA

ev_ebitda_predit  = model_EV_EBITDA.predict ( ev_ebitda_df_V2 )
print ( 'Valeur EV/EBITDA prédite = ', ev_ebitda_predit)

Valeur EV/EBITDA prédite =  [ 6.39912338  5.97201598 40.59211716]


In [148]:
# Stocker la valeur de EV/EBITDA prédite dans une data
#ev_ebitda_predit_df= pd.DataFrame ( { 'EV/EBITDA - Predite' : [6.39912338] } )
#ev_ebitda_predit_df ['EV/EBITDA - Predite'] = ev_ebitda_predit
#ev_ebitda_predit_df

,EV/EBITDA - Predite
0,6.399123


In [40]:
#Pour calculer la moyenne
ev_ebitda_predit = classe_12_secteur_food['EV/EBITDA - Last'].mean ()
ev_ebitda_predit

19.75846153846154

### Utilisation des modèles prédictifs des ratios

In [250]:
#Importation de la data sur laquel les modèles vont etre appliqué
data = pd.read_excel ( 'Data Finpack V2/resultats_test_echantillon/Echantillon détaillé.xlsx', sheet_name = 'Six Flags Entertainment Corp.' )
data

,Société,Secteur,Pays,Description,Mkt. Cap. (kMAD) (Last),E.V (kMAD),EV/EBITDA 2022,EV/EBITDA 2023,EV/EBITDA 2024
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Cedar Fair, L.P.",NaN,USA,NaN,NaN,NaN,7.571019,7.914482,7.275835
2,SeaWorld Entertainment Inc.,NaN,USA,NaN,NaN,NaN,7.278102,6.998377,6.699353
3,Hollywood Bowl Group Plc,NaN,GBR,NaN,NaN,NaN,6.300051,6.420664,6.024327
4,"Tokyotokeiba Co., Ltd.",NaN,JPN,NaN,NaN,NaN,5.537798,5.571403,4.962749
5,"Fuji Kyuko Co., Ltd.",NaN,JPN,NaN,NaN,NaN,NaN,NaN,NaN
6,ROUND ONE Corp.,NaN,JPN,NaN,NaN,NaN,5.547984,4.903133,4.551049
7,"Shenzhen Overseas Chinese Town Co., Ltd.",NaN,CHN,NaN,NaN,NaN,N/M,13.866665,12.501871
8,RCI Hospitality Holdings Inc.,NaN,USA,NaN,NaN,NaN,9.19021,8.948452,7.220814
9,IMAX Corp.,NaN,CAN,NaN,NaN,NaN,14.989339,9.969050,9.291179


In [254]:
#Selectionne de la colonne EV/EBITDA 2022
data_1 = data ['EV/EBITDA 2023']
data_1 = data_1.drop ([ 11,12, 13, 14, 15], axis =0)
data_1

0           NaN
1      7.914482
2      6.998377
3      6.420664
4      5.571403
5           NaN
6      4.903133
7     13.866665
8      8.948452
9      9.969050
10     6.979757
Name: EV/EBITDA 2023, dtype: float64

In [255]:
#Suppression des valeurs nulles
data_2 = data_1.dropna()
data_2

1      7.914482
2      6.998377
3      6.420664
4      5.571403
6      4.903133
7     13.866665
8      8.948452
9      9.969050
10     6.979757
Name: EV/EBITDA 2023, dtype: float64

In [256]:
#Définir le secteur et la classe de l'entreprise
secteur_entreprise = 'Recreational Services'
classe_entreprise = 5
#Codage du secteur 
secteur_entreprise = sector_encoder.transform ([secteur_entreprise])
secteur_entreprise

array([93])

In [257]:
#Les valeurs d'ebitda
data_2.values

array([ 7.91448236,  6.99837678,  6.42066432,  5.57140333,  4.9031334 ,
       13.86666528,  8.94845229,  9.96904953,  6.97975734])

In [258]:
#Input du modèle 
val  = [secteur_entreprise.item(), classe_entreprise, 7.91448236,  6.99837678,  6.42066432,  5.57140333,  4.9031334 ,
       13.86666528,  8.94845229,  9.96904953,  6.97975734]
input = pd.DataFrame ( {'Sector' : [], 'Classe': [], 'EV/EBITDA0': [], 'EV/EBITDA1': [], 'EV/EBITDA2': [],
       'EV/EBITDA3': [], 'EV/EBITDA4': [], 'EV/EBITDA5': [], 'EV/EBITDA6': [],
       'EV/EBITDA7': [], 'EV/EBITDA8': []} )
input.loc [0] = val
input

,Sector,Classe,EV/EBITDA0,EV/EBITDA1,EV/EBITDA2,EV/EBITDA3,EV/EBITDA4,EV/EBITDA5,EV/EBITDA6,EV/EBITDA7,EV/EBITDA8
0,93.0,5.0,7.914482,6.998377,6.420664,5.571403,4.903133,13.866665,8.948452,9.96905,6.979757


In [259]:
#Prédire la valeur EV/EBITDA
valeur_predite = model_EV_EBITDA.predict( input )
valeur_predite

array([8.21897614])

P/E

In [192]:
#Input du modèle 
val  = [secteur_entreprise.item(), classe_entreprise, 9.08011828 , 10.16602332 ,  9.78726798,  7.69744277,  6.75666809,
        6.23199287,  9.08011828,   8.58208461]
input = pd.DataFrame ( {'Sector' : [], 'Classe': [], 'P/E-Last-0': [], 'P/E-Last-1': [], 'P/E-Last-2': [],
       'P/E-Last-3': [], 'P/E-Last-4': [], 'P/E-Last-5': [], 'P/E-Last-6': [],
       'P/E-Last-7': []} )
input.loc [0] = val
input

,Sector,Classe,P/E-Last-0,P/E-Last-1,P/E-Last-2,P/E-Last-3,P/E-Last-4,P/E-Last-5,P/E-Last-6,P/E-Last-7
0,9.0,12.0,9.080118,10.166023,9.787268,7.697443,6.756668,6.231993,9.080118,8.582085


In [193]:
#Prédire la valeur EV/EBITDA
valeur_predite = model_P_E.predict( input )
valeur_predite

array([9.2018091])